# 문제1.
*출처 : 롯데멤버스, L.pay|L.POINT, 제6회 L.POINT Big Data Competition*

### 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
%matplotlib inline

In [10]:
cs = pd.read_csv('L사_고객정보.csv')
gd = pd.read_csv('L사_상품정보.csv')
tr = pd.read_csv('L사_거래정보.csv')

gd.pd_c = gd.pd_c.astype(str).str.zfill(4) # 상품과 거래 데이터의 pd_c 컬럼의 데이터 유형과 자리수가 서로 달라 맞춰줌. 
                                # zfill 함수: 문자열 앞 0으로 채우기. 4자리수.
df = pd.merge(tr, cs).merge(gd, on='pd_c')
df.de_dt = df.de_dt.astype(str).astype('datetime64') # 날짜정보를 담고 있는 de_dt 컬럼의 데이터 유형이 정수이기 때문

df.head()

,clnt_id,trans_id,trans_seq,biz_unit,pd_c,de_dt,de_tm,buy_am,buy_ct,clnt_gender,clnt_age,clac_nm1,clac_nm2,clac_nm3
0,21922,104999,3,A03,0182,2019-09-20,12:41,10900,1,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
1,39423,105124,10,A03,0182,2019-09-20,17:26,21800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
2,39423,89469,2,A03,0182,2019-09-01,03:32,25800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
3,39423,88436,1,A03,0182,2019-08-30,17:15,25800,2,F,50,Chilled Foods,Chilled Instant Foods,Chilled Soups
4,18362,50872,3,A03,0182,2019-07-15,09:46,31600,4,F,40,Chilled Foods,Chilled Instant Foods,Chilled Soups


### 1) ratings_matrx 생성하기
- 행은 clnt_id(고객ID), 열은 clac_nm3(상품 소분류명)로, vales='pd_c'(상품 소분류코드)로 pivoting 수행. 구매가 없는 값(NaN)은 모두 0으로 변환

In [11]:
ratings_matrix = pd.pivot_table(df,index = 'clnt_id', columns = 'clac_nm3', values = 'pd_c',
                               aggfunc = lambda x: 1 if len(x) >= 1 else 0, fill_value=0)
ratings_matrix.head()

clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


### 2) 코사인 유사도 구하기
- cosine_similarity를 사용하여 코사인 유사도를 구하고 데이터프레임 형태로 merchan_sim_df에 저장하세요

In [12]:
# ratings_matrix의 행과 열의 위치를 변경

ratings_matrix_T = ratings_matrix.transpose()
ratings_matrix_T.head()

clnt_id,2,9,12,20,23,24,29,38,40,41,...,72333,72340,72342,72356,72361,72373,72400,72410,72423,72424
clac_nm3,,,,,,,,,,,,,,,,,,,,,
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Accessory Bags,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Accident Prevention Equipment,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adhesive Tapes,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Adhesives,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

merchan_sim = cosine_similarity(ratings_matrix_T, ratings_matrix_T)
merchan_sim_df = pd.DataFrame(data=merchan_sim, index = ratings_matrix.columns, 
                             columns = ratings_matrix.columns)
merchan_sim_df.head()

clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clac_nm3,,,,,,,,,,,,,,,,,,,,,
,1.000000,0.060193,0.0,0.000000,0.000000,0.0,0.0,0.077152,0.0,0.000000,...,0.000000,0.0,0.0,0.037389,0.039820,0.000000,0.038236,0.040161,0.097590,0.0
Accessory Bags,0.060193,1.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.017025,...,0.000000,0.0,0.0,0.045011,0.019175,0.021979,0.000000,0.038678,0.000000,0.0
Accident Prevention Equipment,0.000000,0.000000,1.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,...,0.000000,0.0,0.0,0.015264,0.016256,0.000000,0.000000,0.032791,0.000000,0.0
Adhesive Tapes,0.000000,0.000000,0.0,1.000000,0.018359,0.0,0.0,0.000000,0.0,0.000000,...,0.036298,0.0,0.0,0.054573,0.060889,0.025380,0.031891,0.083741,0.000000,0.0
Adhesives,0.000000,0.000000,0.0,0.018359,1.000000,0.0,0.0,0.000000,0.0,0.012451,...,0.045980,0.0,0.0,0.049379,0.059601,0.000000,0.040398,0.063648,0.025777,0.0


### 3) 특정 상품 유사도 추출
- 'Yogurt Drinks' 상품과 유사도가 가장 높은 상위 5개 상품 추출

In [23]:
merchan_sim_df['Yogurt Drinks'].sort_values(ascending = False)[1:6]

clac_nm3
General Snacks        0.482155
Fresh Milk            0.482012
Chicken Eggs          0.479976
Spoon Type Yogurts    0.477880
Tofu                  0.470498
Name: Yogurt Drinks, dtype: float64

### 4) 전체 상품에 대한 상품 코드 예측
- predict_rating 함수를 정의하고, ratings_pred_matrix에 데이터 프레임 형태로 저장하여 출력하세요.

In [31]:
def predict_rating(ratings_arr, merchan_sim_arr):
    ratings_pred = ratings_arr.dot(merchan_sim_arr)/np.array([np.abs(merchan_sim_arr).sum(axis=1)])
    return ratings_pred

In [32]:
ratings_pred = predict_rating(ratings_matrix.values, merchan_sim_df.values)
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index = ratings_matrix.index,
                                  columns = ratings_matrix.columns)
ratings_pred_matrix.head()

clac_nm3,,Accessory Bags,Accident Prevention Equipment,Adhesive Tapes,Adhesives,Adult's Bed Covers and Skirts,Adult's Diapers,Adult's Disposable Briefes,Adults' Bed Fillings,Adults' Bedding Sets,...,Yoga / Pilates Clothing,Yoga Mats / Exercise Mats,Yoghurts / Chunggukjang Makers,Yogurt Drinks,Young Pumpkins,Young Radishes,Yuzus / Quinces,Zipper Poly Bags / Plastic Bags,kelp,life Vests / Safety Accessories
clnt_id,,,,,,,,,,,,,,,,,,,,,
2,0.012300,0.009045,0.007467,0.008229,0.010892,0.015854,0.006684,0.010192,0.009248,0.011271,...,0.012163,0.004203,0.008294,0.016102,0.015277,0.011604,0.010132,0.012060,0.010493,0.013583
9,0.052859,0.050875,0.034678,0.049847,0.050869,0.036422,0.040643,0.065125,0.034191,0.051500,...,0.033866,0.054011,0.067044,0.082844,0.086547,0.069561,0.063618,0.063606,0.075088,0.063010
12,0.002020,0.001669,0.001439,0.001986,0.001951,0.000617,0.001275,0.002437,0.001893,0.002202,...,0.002088,0.002027,0.000000,0.003153,0.003151,0.002867,0.002699,0.002555,0.002565,0.002677
20,0.046240,0.038304,0.023695,0.037119,0.038379,0.017426,0.033944,0.059805,0.069159,0.045758,...,0.028296,0.031759,0.000000,0.065010,0.075219,0.067278,0.047674,0.053412,0.072009,0.048463
23,0.067617,0.052122,0.053685,0.055207,0.057076,0.034219,0.049280,0.063280,0.048946,0.063546,...,0.050503,0.052112,0.010294,0.094091,0.104342,0.087819,0.066717,0.072931,0.096197,0.074502


### 5) 전체 상품에 대한 상품 코드 예측 평가
- get_mse 함수를 생성하여, 전체 상품에 대한 mse를 구하세요.

In [33]:
from sklearn.metrics import mean_squared_error

def get_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)

print('아이템 기반 모든 인접 이웃 MSE: ', get_mse(ratings_pred, ratings_matrix.values))

아이템 기반 모든 인접 이웃 MSE:  0.7433403825371361


## 문제 2. 특정 상품과 가장 비슷한 유사도를 가지는 영화에 대해서만 유사도 벡터 적용 (1번과 연결)

### 1) 특정 상품과 가장 비슷한 유사도를 가지는 상품을 10개 나열하는 함수 만들기.

In [ ]:
def predict_rating_topsim():
    

### 2) 위 문제에서 만든 함수를 이용해 예측 구매율을 계산하고, 실제 구매율과의 MSE 구하기.

### 3) 고객번호 9번에 대해 상품 추천하기 (clnt_id=9)
- clnt_id =9 가 구매한 상품 상품 나열
- 고객이 구매하지 않은 상품을 리스트 객체로 반환하는 함수 생성 
- 위에서 만든 두 함수를 이용해 최종적으로 고객에게 상품을 추천하는 함수 만들기.

### 4) 행렬분해를 이용해 잠재 요인 협업 필터링 직접 구현하기.
- SGD기법을 사용하는 행렬 분해 함수 만들기
- 고객-상품 구매 행렬 다시 만들기
- 만들어진 행렬을 행렬분해하기 (steps=200, 잠재 요인 차원 = 30, 학습률과 L2 Regularization 계수 = 0.01)
- 9번 고객에 대해 

## 문제3) Surprise를 활용한 책 추천 시스템 구축

In [1]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
import pandas as pd
from surprise import Reader, Dataset

### 3-1) book_ratings 데이터를 불러온 후, 아래의 형식과 같이 출력해주세요.
- Total number of ratings : xxx
- Number of unique users : xxx
- Number of unique movies : xxx

In [2]:
ratings=pd.read_csv('book_ratings.csv')
ratings.head()

,user_id,book_id,rating
0,1,1180,4
1,1,4893,3
2,1,6285,4
3,2,8034,4
4,2,8855,5


### books 데이터를 불러온 후, id, authors, title에 해당하는 열만 추출하여 books에 저장해주세요

In [3]:
books=pd.read_csv("books.csv",encoding='latin1')
books

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,Unnamed: 23
0,1,2767052,2767052,2792775,272,439023483,9.780000e+12,Suzanne Collins,2008,The Hunger Games,...,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...,NaN
1,2,3,3,4640799,491,439554934,9.780000e+12,"J.K. Rowling, Mary GrandPrÃ©",1997,Harry Potter and the Philosopher's Stone,...,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,NaN
2,3,41865,41865,3212258,226,316015849,9.780000e+12,Stephenie Meyer,2005,Twilight,...,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...,NaN
3,4,2657,2657,3275794,487,61120081,9.780000e+12,Harper Lee,1960,To Kill a Mockingbird,...,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...,NaN
4,5,4671,4671,245494,1356,743273567,9.780000e+12,F. Scott Fitzgerald,1925,The Great Gatsby,...,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9990,9996,7130616,7130616,7392860,19,441019455,9.780000e+12,Ilona Andrews,2010,Bayou Moon,...,18856,1180,105,575,3538,7860,6778,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460s...,NaN
9991,9997,208324,208324,1084709,19,067973371X,9.780000e+12,Robert A. Caro,1990,Means of Ascent,...,12952,395,303,551,1737,3389,6972,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...,NaN
9992,9998,77431,77431,2393986,60,039330762X,9.780000e+12,Patrick O'Brian,1977,The Mauritius Command,...,10733,374,11,111,1191,4240,5180,https://images.gr-assets.com/books/1455373531m...,https://images.gr-assets.com/books/1455373531s...,NaN
9993,9999,8565083,8565083,13433613,7,61711527,9.780000e+12,Peggy Orenstein,2011,Cinderella Ate My Daughter: Dispatches from th...,...,11994,1988,275,1002,3765,4577,2375,https://images.gr-assets.com/books/1279214118m...,https://images.gr-assets.com/books/1279214118s...,NaN


### 3-2) Ratings 데이터프레임에서 surprise 데이터 세트로 로딩하여 RMSE를 출력해주세요
- 알고리즘은 SVD, n_factors=30
- test_size=0.25, random_state=0
- movies_ratings_small에서 rating의 범위를 기반으로 rating_scale 설정

In [ ]:
import pandas as pd
from surprise import Reader, Dataset

### 3-3) GridSearchCV를 이용하여 최적 파라미터를 산출하세요
- param_grid={'n_epochs':[15,20,25],'n_factors':[50,80,100,120]}

In [ ]:
from surprise.model_selection import GridSearchCV

### 3-4) DatasetAutoFolds를 활용하여 개인 책 추천 시스템을 구축하세요

- 1)book_ratings에서 헤드를 제거하여 새로운 파일을 생성해주세요. (book_ratings_noh.csv)

- 2) book_ratings_noh 파일 기반으로 전체 데이터를 SVD로 학습 시켜주세요. (n_epochs와 n_factors는 GridSearchCV를 기반으로 나온 최적 파라미터, random_state=0)

In [ ]:
from surprise.dataset import DatasetAutoFolds

- 교재에서 사용한 get_unseen_surprise와 recomm_movie_by_surprise 함수를 기반으로 userID=10인 사용자에게 top10 개의 책 리스트와 예측 평점을 산출하세요.